# Base Line

## Step 0: Data Processing

In [3]:
import tensorflow as tf
import numpy as np
import pickle

In [4]:
def unpickle(file):
   
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [5]:
batch_1 = unpickle("data\cifar-10-batches-py\data_batch_1")
x_train = batch_1[b'data'] 
y_train = batch_1[b'labels']

In [16]:
y_train0 = np.empty((0,10), int)
for y in y_train:
    indi= [1 if i==y else 0 for i in range(0,10)]
    y_train0 = np.append(y_train0, np.array([indi]), axis=0)

numpy.ndarray

## Step1: Functions Initiation

In [17]:
def perf_eval(y_pred, y_true):
    """a function to evaluate performance of predicted y values vs true class labels"""
    # now look at some data
    print(' sample pred: {0}\n sample true: {1}'.format(np.argmax(y_pred[0:20],1),np.argmax(y_true[0:20],1)))
    # avg accuracy
    is_correct_vals = np.equal(np.argmax(y_pred,1),np.argmax(y_true,1))
    accuracy_vals = np.mean(is_correct_vals)
    print(' mean classification accuracy: {0}%'.format(100*accuracy_vals))
    # Dig in a little deeper.  Where did we make correct predictions?  Does this seem reasonable?
    print(' correct predictions by class: {0}'.format(y_true[is_correct_vals,:].sum(axis=0)))

## Step2:Build model

In [18]:
tf.reset_default_graph()
# placeholders for data
x = tf.placeholder(tf.float32, shape=[None, 32*32*3])
y = tf.placeholder(tf.float32, shape=[None, 10]) 
# variables to be optimized
W = tf.Variable(tf.zeros([32*32*3,10]))
b = tf.Variable(tf.zeros([10]))

In [19]:
# how to predict label from data
y_model = tf.nn.softmax(tf.matmul(x,W) + b)
# the objective function
cross_ent = tf.reduce_mean(-tf.reduce_sum(y*tf.log(y_model), reduction_indices=[1]))
# the cost function to be optimized
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_ent)
# performance quantification
correct_pred = tf.equal(tf.argmax(y_model,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

## Step3: Run the model

In [20]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # train model
    for i in range(1001):
        #batch = mnist.train.next_batch(50)
        train_step.run(feed_dict={x: x_train, y: y_train})
        # print diagnostics
        print(".", end='', flush=True)
        if i%100 == 0:
            train_error = sess.run(accuracy, {x: x_train, y: y_train})
            print("\rAfter step {0:3d}, training accuracy {1:0.4f}".format(i, train_error), flush=True)
        #if i%1000 == 0: 
            #test_error = sess.run(accuracy, {x:mnist.test.images, y:mnist.test.labels})
            #print("\rAfter step {0:3d}, test accuracy {1:0.4f}".format(i, test_error), flush=True)

    # evaluate newly trained model on training data
    #y_pred_values = sess.run(y_model, feed_dict={x: mnist.test.images})
   # W_val = sess.run(W)
    #b_val = sess.run(b)
    #f_val = sess.run(tf.matmul(x,W) + b, feed_dict={x: X_train[some_digit,:].reshape([1,784])})

ValueError: Cannot feed value of shape (10000,) for Tensor 'Placeholder_1:0', which has shape '(?, 10)'